In [15]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

In [3]:
sql = '''
SELECT *
FROM dividends
LIMIT 1'''
struct = pd.read_sql(sql, conpf)
struct.dtypes

id            int64
stock_id      int64
name         object
year          int64
quarter       int64
number        int64
ppu         float64
amt         float64
net         float64
x_date       object
p_date       object
dtype: object

In [16]:
sql = '''
SELECT YEAR(p_date) AS p_year, MONTH(p_date) AS p_month, name, year, quarter, number, ppu, amt, net, x_date, p_date
FROM dividends
WHERE YEAR(p_date) = 2021
'''
dtl_by_month = pd.read_sql(sql, conpf)
dtl_by_month.shape

(106, 11)

In [17]:
ttl_by_month = dtl_by_month.groupby(['p_year','p_month','name'], as_index=True).agg(
    {
        'net':['sum','count'],
    }
)
ttl_by_month

net      
                            sum count
p_year p_month name                  
2021   1       ORI      5670.00     1
       2       LPF      1231.65     1
       3       CPTGF    1620.00     1
               DIF     20880.00     1
               EGATIF   2000.00     1
...                         ...   ...
       12      MCS     25200.00     1
               POPF     2223.00     1
               RCL      6075.00     1
               RJH      2970.00     1
               STA     16875.00     1

[105 rows x 2 columns]

In [18]:
ttl_by_month.sum()

net  sum      700874.5
     count       106.0
dtype: float64

In [19]:
ttl_by_month.groupby(level='p_month').sum()

net      
               sum count
p_month                 
1          5670.00     1
2          1231.65     1
3         71491.44     9
4        105808.20    12
5        121199.76    21
6         77048.75    10
7          1350.00     1
8          9180.00     3
9        191048.50    31
10         8550.00     2
11         1800.00     1
12       106496.20    14

In [35]:
sql = '''
SELECT name, p_date, FORMAT(amt,2) AS amt, FORMAT(net,2) AS net, format(amt-net,2) AS refund, year, quarter AS q
FROM dividends
WHERE YEAR(p_date) = 2021
ORDER BY p_date DESC'''
dtl_by_month = pd.read_sql(sql, conpf)
dtl_by_month

,name,p_date,amt,net,refund,year,q
0,HREIT,2021-12-23,"1,738.00","1,564.20",173.80,2021,3
1,EGATIF,2021-12-17,640.00,640.00,0.00,2021,3
2,EGATIF,2021-12-17,"1,040.00","1,040.00",0.00,2021,3
3,MCS,2021-12-09,"25,900.00","25,200.00",700.00,2021,3
4,IVL,2021-12-09,"5,250.00","4,725.00",525.00,2021,3
...,...,...,...,...,...,...,...
98,DIF,2021-03-12,"20,880.00","20,880.00",0.00,2020,4
99,MC,2021-03-11,"3,500.00","3,150.00",350.00,2020,2
100,GVREIT,2021-03-10,"3,666.60","3,299.94",366.66,2020,4
101,LPF,2021-02-15,"1,368.50","1,231.65",136.85,2020,3


In [11]:
sql = '''
SELECT YEAR(p_date) AS pay_year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
GROUP BY pay_year
ORDER BY pay_year DESC'''
ttl_by_year = pd.read_sql(sql, conpf)
ttl_by_year

,pay_year,grs_amt,net_amt,refund
0,2021,739665.10,689609.50,50055.60
1,2020,616228.94,576030.84,40198.10
2,2019,552424.34,510266.46,42157.88
3,2018,351895.60,309725.58,42170.02
4,2017,245981.10,219514.23,26466.87


In [28]:
dtl_by_month[mask].count()

month    13
name     13
net      13
dtype: int64

In [5]:
sql = '''
SELECT name, COUNT(*) AS qtrs, SUM(net) AS ttl_net
FROM dividends
GROUP BY name
ORDER BY SUM(net) DESC'''
total = pd.read_sql(sql, conpf)
total.head(30)

,name,qtrs,ttl_net
0,JASIF,15,325800.00
1,MCS,10,288180.00
2,DIF,11,174689.50
3,NOBLE,4,100980.00
4,CPNREIT,9,100219.10
5,SAT,5,96040.58
6,FSMART,7,80244.00
7,ORI,9,79245.00
8,LH,7,78165.00
9,TISCO,3,69390.00


### Summarized profit by name

In [5]:
sql = '''
SELECT name, sum(profit) AS profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
GROUP BY stocks.name
ORDER BY sum(profit) DESC LIMIT 10'''
profits_by_stock = pd.read_sql(sql, conpf)
profits_by_stock

,name,profit
0,CPNREIT,545770.06
1,THANI,237046.32
2,BEAUTY,192974.08
3,JMT,175955.98
4,MTC,147177.71
5,ASIAN,141685.61
6,TU,132108.09
7,STA,130902.48
8,SAWAD,115506.59
9,SIS,99705.63


In [6]:
sql = '''
SELECT YEAR(date) AS year, MONTH(date) AS month, profit
FROM sells 
WHERE YEAR(date) = 2021
ORDER BY YEAR(date) DESC, MONTH(date) DESC
'''
sells = pd.read_sql(sql, conpf)
sells

,year,month,profit
0,2021,12,-2827.02
1,2021,12,-474.44
2,2021,12,10993.89
3,2021,12,11991.68
4,2021,12,8730.89
...,...,...,...
241,2021,1,25560.10
242,2021,1,12524.91
243,2021,1,17713.38
244,2021,1,7352.16


In [7]:
grouped = sells.groupby(['year','month'])
grouped

In [8]:
profit_by_month = grouped['profit'].sum()
profit_by_month

year  month
2021  1         51296.67
      2        202547.56
      3        -19735.49
      4         65267.16
      5         79180.57
      6         14606.32
      7       -102107.17
      8         69193.08
      9          1544.44
      10       -10446.93
      11       119130.67
      12        90294.23
Name: profit, dtype: float64

In [9]:
grouped.agg(['sum','mean', 'max', 'min','count'])

profit                                        
                  sum          mean       max       min count
year month                                                   
2021 1       51296.67   8549.445000  26592.67 -38446.55     6
     2      202547.56  20254.756000  40643.41    236.43    10
     3      -19735.49   -704.838929  21118.92 -14043.21    28
     4       65267.16   4661.940000  16041.52  -7426.10    14
     5       79180.57   2932.613704  28913.18 -22633.46    27
     6       14606.32    635.057391  28634.54 -24598.02    23
     7     -102107.17  -3646.684643  37641.19 -26836.72    28
     8       69193.08   2096.760000  29056.45 -34941.06    33
     9        1544.44     44.126857  21638.96  -9697.69    35
     10     -10446.93  -1741.155000    734.87 -12550.51     6
     11     119130.67   7007.686471  23238.08 -10455.17    17
     12      90294.23   4752.327895  12616.82 -16057.05    19

### Temporary process to calculate dividend portion of profit

In [10]:
year = 2020
quarter = 4

In [11]:
sql = """
SELECT name, year, quarter, aq_eps, ay_eps
FROM epss 
WHERE year = %s AND quarter = %s
"""
sql = sql % (year, quarter)
df_epss = pd.read_sql(sql, conlt)
df_epss.head()

,name,year,quarter,aq_eps,ay_eps
0,OISHI,2020,4,2.8500,3.2900
1,TFFIF,2020,4,0.2771,2.2000
2,GVREIT,2020,4,0.9170,1.0902
3,IRC,2020,4,1.1200,0.8300
4,AOT,2020,4,0.3000,1.7500


In [12]:
sql = '''
SELECT name, dividend
FROM dividend 
'''
df_dividend = pd.read_sql(sql, const)
df_dividend

,name,dividend
0,WHART,0.8298
1,QH,0.1100
2,LH,0.5500
3,MCS,1.3700
4,TQM,2.6000
...,...,...
79,BGRIM,0.4500
80,KBANK,3.0000
81,MC,0.5500
82,RJH,1.7000


In [13]:
df_merge = pd.merge(df_epss, df_dividend, on='name', how='inner')
df_merge['xxx_pct'] = df_merge.dividend / df_merge.aq_eps * 100
df_merge.set_index('name',inplace=True)
df_merge.sort_values(['xxx_pct'],ascending=[True])

,year,quarter,aq_eps,ay_eps,dividend,xxx_pct
name,,,,,,
TOP,2020,4,-1.620,3.080,1.30,-80.246914
BCP,2020,4,-5.500,1.180,1.40,-25.454545
KBANK,2020,4,12.420,16.180,3.00,24.154589
DOHOME,2020,4,0.340,0.340,0.10,29.411765
ROJNA,2020,4,0.680,0.910,0.20,29.411765
...,...,...,...,...,...,...
PTT,2020,4,1.320,3.200,2.02,153.030303
RCL,2020,4,2.105,-0.593,3.50,166.270784
TMT,2020,4,0.620,0.260,1.10,177.419355


In [14]:
df_merge.loc['MCS']

year        2020.00
quarter        4.00
aq_eps         2.00
ay_eps         1.28
dividend       1.37
xxx_pct       68.50
Name: MCS, dtype: float64